In [1]:
import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(gpu, True)
import keras_tuner as kt

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf

class DataLoader:
    """
    The data loader class is used to load data from a directory and generate batches of data.
    """
    def __init__(self, shape, batch_size, scale = True):
        """
        Initializes the data loader class.
        Args:
            shape: shape of the image
            batch_size: size of the batch
            scale: whether to scale the image to the range [0, 1]
        """
        self.indices = {}
        self.__file_names__ = []
        self.__labels__ = []
        self.__shape__ = shape
        self.__batch_size__ = batch_size
        self.__scale__ = scale
    
    def load_from_directory(self, folder_directory):
        """
        Loads data from a given directory.
        Args:
            folder_directory: directory of the data
        Returns:
            None
        """
        self.num_files = 0
        classes = os.listdir(folder_directory)
        for i in range(len(classes)):
            sub_dir = os.path.join(folder_directory, classes[i])
            filename = os.listdir(sub_dir)
            for file_ in filename:
                file_path = os.path.join(sub_dir, file_)
                self.__file_names__.append(file_path)
                self.num_files += 1
                self.__labels__.append(i)
            self.indices[i] = classes[i]
        print(f"Loaded {self.num_files} files of {len(classes)} classes")
        return None

    def data_generator(self, shape = (360,360), batch_size = 4, scale = True):
        """
        generator for the data loader class.
        Args:
            shape: shape of the image
            batch_size: size of the batch
            scale: whether to scale the image to the range [0, 1]

        Returns:
            batch of images and labels
        """
        self.__shape__ = shape
        self.__batch_size__ = batch_size
        self.__scale__ = scale

        if len(self.__file_names__) == 0:
            raise Exception("No data loaded")
        else:
            while True:
                batch_y_train = {}
                batch_x = np.zeros(shape = (self.__batch_size__, self.__shape__[0], self.__shape__[1], 1))
                batch_y = np.zeros(shape = (self.__batch_size__))
                for i in range(self.__batch_size__):
                    index = np.random.randint(0, len(self.__file_names__))
                    img = cv2.imread(self.__file_names__[index])
                    img = cv2.resize(img, self.__shape__)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                    if self.__scale__:
                        img = img / 255.0 
                    img = np.expand_dims(img, axis = -1) # add a channel dimension
                    batch_x[i] = img
                    batch_y[i] = self.__labels__[index]
                y = tf.one_hot(batch_y, depth = len(self.indices))
                for i in self.indices.keys():
                    batch_y_train[self.indices[i]] = y[:, i]
                yield batch_x, batch_y_train

In [3]:
BATCH_SIZE = 4

data_loader = DataLoader(shape=(640,640), batch_size = BATCH_SIZE) 
data_loader.load_from_directory("/home/mugesh/IB/DocuNet/data/processed_data/")
data_gen = data_loader.data_generator(shape = (640,640), batch_size = BATCH_SIZE)

Loaded 696 files of 4 classes


In [4]:
import os
import numpy as np
import cv2

In [5]:
def build_model(hp):
    input_ = tf.keras.layers.Input(shape=(640,640,1), name ="input")

    conv_1 = tf.keras.layers.Conv2D(filters = hp.Int('conv_1', min_value=32, max_value=64, step=16)
    , kernel_size = (3,3), padding = "same", name="conv_1")(input_)
    act_1 = tf.keras.layers.Activation('relu', name='act_1')(conv_1)
    pool_1 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_1")(act_1)

    conv_2 = tf.keras.layers.Conv2D(filters = hp.Int('conv_2', min_value=32, max_value=64, step=16)
    , kernel_size = (3,3), padding = "same" , name="conv_2")(pool_1)
    act_2 = tf.keras.layers.Activation('relu', name='act_2')(conv_2)
    pool_2 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_2")(act_2)

    conv_3 = tf.keras.layers.Conv2D(filters = hp.Int('conv_3', min_value=16, max_value=64, step=16)
    , kernel_size = (3,3), padding = "same",  name="conv_3")(pool_2)
    act_3 = tf.keras.layers.Activation('relu', name='act_3')(conv_3)
    pool_3 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_3")(act_3)

    conv_4 = tf.keras.layers.Conv2D(filters = hp.Int('conv_4', min_value=16, max_value=64, step=16) 
    , kernel_size = (3,3), padding = "same", name="conv_4")(pool_3)
    act_4 = tf.keras.layers.Activation('relu', name='act_4')(conv_4)
    pool_4 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_4")(act_4)
    pool_5 = tf.keras.layers.MaxPool2D(pool_size = (2,2), name = "pool_5")(pool_4)

    flatten = tf.keras.layers.Flatten(name="flatten")(pool_5)


    dense_1 = tf.keras.layers.Dense(units= hp.Int('dense_1', min_value=16, max_value=96, step=16), name="dense_1")(flatten)
    dense_2 = tf.keras.layers.Dense(units= hp.Int('dense_2', min_value=16, max_value=96, step=16), name="dense_2")(dense_1)

    shaded = tf.keras.layers.Dense(1,activation='sigmoid', name="shaded")(dense_2)
    gridline = tf.keras.layers.Dense(1,activation='sigmoid', name="gridline")(dense_2)
    good = tf.keras.layers.Dense(1,activation='sigmoid', name="good")(dense_2)
    black_border = tf.keras.layers.Dense(1,activation='sigmoid', name="black_border")(dense_2)

    model_outputs = [black_border, good, gridline, shaded]
    model = tf.keras.models.Model(input_, model_outputs, name ="model_v1")


    model.compile(
    loss={
        "shaded": 'binary_crossentropy',
        "gridline": "binary_crossentropy",
        "good": "binary_crossentropy",
        "black_border": "binary_crossentropy"
    },
    optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4])),
    metrics=['accuracy']
    )
    return model

In [ ]:
# tuner = kt.RandomSearch(build_model, objective=kt.Objective("loss", direction = "min")
# , max_trials= 15 , directory='keras_tuner', project_name='RandomSearch')

In [ ]:
tuner = kt.BayesianOptimization(build_model, objective=kt.Objective("loss", direction = "min"), max_trials= 15 , directory='keras_tuner', project_name='BayesianOptimization')

In [ ]:
tuner.search(    data_gen,
    steps_per_epoch= data_loader.num_files // BATCH_SIZE,
    epochs = 3
) 

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
tuner.get_best_models()[0].save("bayesian_best.h5")